In [545]:
import sqlite3
import pandas as pd
import numpy as np

In [546]:
conn = sqlite3.connect('SDOH.db')
finalanswers_table = 'FinalAnswers'
main_query = f'SELECT * FROM {finalanswers_table}'

In [547]:
df = pd.read_sql_query(main_query, conn)

In [548]:
df['8261'].fillna(1, inplace = True)
df['1570008806'].fillna(1, inplace = True)
df['1570008806'] = df['1570008806'].astype(int)
df['1572879828'].fillna(1, inplace = True)

In [549]:
df.fillna(0, inplace = True)

In [550]:
assistance_columns = df.columns[35:48]
df['14208:None'] = df.apply(lambda row: 1 if (row[assistance_columns] == 0).all() else row['14208:None'], axis = 1)

In [551]:
df

,ID,8260,8261,8306,8307,13895,1570008805,1570008806,1570008807,1572879810,...,14208:Stress,14208:SocialConnections,14208:PhysicalActivity,14208:None,1572879830:Married,1572879830:LivingWithPartner,1572879830:NeverMarried,1572879830:Widowed,1572879830:Separated,1572879830:Divorced
0,100439,0.0,1.0,1.0,1.0,1.0,0.0,2,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,100468,0.0,1.0,1.0,1.0,1.0,0.0,1,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,100581,0.0,1.0,1.0,1.0,1.0,0.0,1,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,100765,0.0,1.0,1.0,1.0,1.0,0.0,1,0.0,3.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,101013,0.0,1.0,1.0,1.0,1.0,0.0,2,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,96957,0.0,1.0,1.0,1.0,1.0,0.0,1,0.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
513,97364,0.0,1.0,1.0,1.0,0.0,0.0,1,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
514,97474,0.0,1.0,1.0,1.0,0.0,1.0,1,0.0,2.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
515,98710,0.0,1.0,2.0,0.0,0.0,0.0,1,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [552]:
scoring_df = df[['ID']].copy()

In [553]:
scoring_df['Food'] = df.apply(lambda row: ((row['1572879820'] - 1) if row['1572879820'] != 0 else 0) + 
                                          ((row['1572879821'] - 1) if row['1572879821'] != 0 else 0), axis = 1)

In [554]:
scoring_df['Transportation'] = df.apply(lambda row: row['1572879817'] + row['1572879818'], axis = 1)

In [555]:
scoring_df['InternetAccess'] = df.apply(lambda row: (1 if row['8261'] == 0 else 0) + (((row['8306'] - 1) * 0.5) if row['8306'] != 0 else 0), axis = 1)

In [556]:
scoring_df['HousingStability'] = df.apply(lambda row: (row['1570008805'] * 3) + 
                                                     (1 if row['1570008806'] == 2 else
                                                      2 if row['1570008806'] == 3 else
                                                      3 if (row['1570008806'] == 0 or row['1570008806'] in range(4, 51))
                                                      else 0) +
                                                     (row['1570008807'] * 6) +
                                                     ((((row['8260'] - 1) * 0.25) * 3) if row['8260'] != 0 else 0),
                                                     axis = 1)

In [557]:
scoring_df['HousingProblems'] = df.apply(lambda row: row['8258:Pests'] + row['8258:Mold'] + row['8258:LeadPaint/Pipes'] +
                                                     row['8258:WaterLeaks'] + row['8258:SmokeDetectors'] + row['8258:Oven/Stove'] +
                                                     row['8258:HeatLack'] +
                                                     (((row['8307'] - 1) * 2) if row['8307'] != 0 else 0),
                                                       axis = 1)

In [558]:
scoring_df['PersonalSafety'] = df.apply(lambda row: row['1572879813'] + row['1572879812'] + row['1572879815'] + row['1572879814'], axis = 1)

In [559]:
scoring_df['FinancialStability'] = df.apply(lambda row: (row['1572879810'] - 1) if row['1572879810'] != 0 else 0, axis = 1)

In [560]:
scoring_df['Stress'] = df.apply(lambda row: (row['1572879823'] - 1) if row['1572879823'] != 0 else 0, axis = 1)

In [561]:
scoring_df['SocialConnections'] = df.apply(lambda row: (((row['1572879825'] - 1) * 0.5) if row['1572879825'] != 0 else 0) +
                                                       ((((row['1572879826'] - 1) * 0.25) * 3) if row['1572879826'] != 0 else 0) +
                                                       (((row['1572879827'] - 1) * 0.5) if row['1572879827'] != 0 else 0) +
                                                       (0.5 if row['1572879828'] == 0 else 0) +
                                                       (((row['1572879829'] - 1) * 0.25) if row['1572879829'] != 0 else 0),
                                                       axis = 1)

In [562]:
scoring_df['PhysicalActivity'] = df.apply(lambda row: row['1572879832'] + row['1572879833'], axis = 1)

In [563]:
max_values = {
    'Food' : 4,
    'Transportation' : 2,
    'InternetAccess' : 2,
    'HousingStability' : 13.5,
    'HousingProblems' : 11,
    'PersonalSafety' : 4,
    'FinancialStability' : 4,
    'Stress' : 4,
    'SocialConnections' : 7,
    'PhysicalActivity' : 6
}

In [564]:
request_assistance_df = df.iloc[:, [0] + list(range(35, 48))]

In [565]:
request_assistance_column_names = ['ID', 'Food', 'Transportation', 'InternetAccess', 'HousingStability', 'HousingProblems',
                                   'Utilities', 'FinancialStability', 'Employment', 'Education', 'Stress',
                                   'SocialConnections', 'PhysicalActivity', 'None']
request_assistance_df.columns = request_assistance_column_names

In [566]:
scaled_df = scoring_df.copy()
for category in max_values:
    scaled_df[category] = (scaled_df[category] / max_values[category]) * 100

In [567]:
importance_levels = {
    'Food' : 3,
    'Transportation' : 2,
    'InternetAccess' : 1,
    'HousingStability' : 3,
    'HousingProblems' : 2,
    'PersonalSafety' : 2.5,
    'FinancialStability' : 3,
    'Stress' : 2,
    'SocialConnections' : 2,
    'PhysicalActivity' : 1.5
}

In [568]:
model_df = scaled_df.copy()
for category in importance_levels:
    model_df[category] = (model_df[category] * importance_levels[category]).round(1)

In [569]:
def boost_scores(row):
    current_id = row['ID']
    for category in max_values:
        if category == 'PersonalSafety':
            continue
        if row[category] == 1:
            model_df.loc[model_df['ID'] == current_id, category] += 100

In [570]:
request_assistance_df.apply(boost_scores, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
512    None
513    None
514    None
515    None
516    None
Length: 517, dtype: object

In [571]:
prioritized_categories = {}
for index, row in model_df.iterrows():
    patient_id = row['ID']
    categories = row.drop('ID')
    sorted_categories = categories[categories > 0].sort_values(ascending = False).index.tolist()
    prioritized_categories[patient_id] = sorted_categories

In [572]:
list_of_dicts = []
for patient_id, ranks in prioritized_categories.items():
    patient_dict = {'ID' : patient_id}
    for i, rank in enumerate(ranks, 1):
        patient_dict[f'Rank{i}'] = rank
    list_of_dicts.append(patient_dict)

In [573]:
priority_df = pd.DataFrame(list_of_dicts)
priority_df

,ID,Rank1,Rank2,Rank3,Rank4,Rank5,Rank6,Rank7,Rank8,Rank9,Rank10
0,100439,SocialConnections,Stress,FinancialStability,PhysicalActivity,HousingStability,NaN,NaN,NaN,NaN,NaN
1,100468,PhysicalActivity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100581,Stress,PhysicalActivity,SocialConnections,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100765,Stress,SocialConnections,Food,FinancialStability,PhysicalActivity,NaN,NaN,NaN,NaN,NaN
4,101013,FinancialStability,PhysicalActivity,SocialConnections,Stress,HousingStability,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
512,96957,FinancialStability,SocialConnections,PhysicalActivity,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,97364,Transportation,FinancialStability,Stress,SocialConnections,PhysicalActivity,Food,HousingProblems,NaN,NaN,NaN
514,97474,Stress,FinancialStability,HousingStability,PhysicalActivity,SocialConnections,NaN,NaN,NaN,NaN,NaN
515,98710,FinancialStability,SocialConnections,PhysicalActivity,Stress,InternetAccess,NaN,NaN,NaN,NaN,NaN


In [574]:
df.to_sql('FinalAnswersNoNull', conn, if_exists = 'replace', index = False)
model_df.to_sql('ScoringMechanism', conn, if_exists = 'replace', index = False)
request_assistance_df.to_sql('RequestAssistanceScores', conn, if_exists = 'replace', index = False)
priority_df.to_sql('SortedCategories', conn, if_exists = 'replace', index = False)
scoring_df.to_sql('RawScoresQuestionsOnly', conn, if_exists = 'replace', index = False)
scaled_df.to_sql('ScaledScoresQuestionsOnly', conn, if_exists = 'replace', index = False)

517

In [575]:
conn.commit()
conn.close()